# An Introduction to LlamaIndex Query Pipelines

## Overview
LlamaIndex provides a declarative query API that allows you to chain together different modules in order to orchestrate simple-to-advanced workflows over your data.

This is centered around our `QueryPipeline` abstraction. Load in a variety of modules (from LLMs to prompts to retrievers to other pipelines), connect them all together into a sequential chain or DAG, and run it end2end.

**NOTE**: You can orchestrate all these workflows without the declarative pipeline abstraction (by using the modules imperatively and writing your own functions). So what are the advantages of `QueryPipeline`? 

- Express common workflows with fewer lines of code/boilerplate
- Greater readability
- Greater parity / better integration points with common low-code / no-code solutions (e.g. LangFlow)
- [In the future] A declarative interface allows easy serializability of pipeline components, providing portability of pipelines/easier deployment to different systems.

## Cookbook

In this cookbook we give you an introduction to our `QueryPipeline` interface and show you some basic workflows you can tackle.

- Chain together prompt and LLM
- Chain together query rewriting (prompt + LLM) with retrieval
- Chain together a full RAG query pipeline (query rewriting, retrieval, reranking, response synthesis)
- Setting up a custom query component

## Setup

Here we setup some data + indexes (from PG's essay) that we'll be using in the rest of the cookbook.

In [1]:
# setup Arize Phoenix for logging/observability
import phoenix as px

px.launch_app()
import llama_index

llama_index.set_global_handler("arize_phoenix")

🌍 To view the Phoenix app in your browser, visit http://127.0.0.1:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [2]:
from llama_index.query_pipeline.query import QueryPipeline
from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    SimpleDirectoryReader,
    load_index_from_storage,
)

In [3]:
reader = SimpleDirectoryReader("../data/paul_graham")

In [4]:
docs = reader.load_data()

In [5]:
import os
from llama_index.storage import StorageContext

if not os.path.exists("storage"):
    index = VectorStoreIndex.from_documents(docs)
    # save index to disk
    index.set_index_id("vector_index")
    index.storage_context.persist("./storage")
else:
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage")
    # load index
    index = load_index_from_storage(storage_context, index_id="vector_index")

## 1. Chain Together Prompt and LLM 

In this section we show a super simple workflow of chaining together a prompt with LLM.

We simply define `chain` on initialization. This is a special case of a query pipeline where the components are purely sequential, and we automatically convert outputs into the right format for the next inputs.

In [6]:
# try chaining basic prompts
prompt_str = "Please generate related movies to {movie_name}"
prompt_tmpl = PromptTemplate(prompt_str)
llm = OpenAI(model="gpt-3.5-turbo")

p = QueryPipeline(chain=[prompt_tmpl, llm], verbose=True)

In [7]:
output = p.run(movie_name="The Departed")

> Running module 4e7cc59d-b05e-4f0b-b45a-c6520bc5c959 with input: 
movie_name: The Departed

> Running module e6a152de-a038-4a0a-a130-553b76772a3f with input: 
messages: Please generate related movies to The Departed



In [8]:
print(str(output))

assistant: 1. Infernal Affairs (2002) - The Departed is actually a remake of this Hong Kong crime thriller, which follows a similar storyline of undercover cops infiltrating the criminal underworld.

2. The Town (2010) - Directed by Ben Affleck, this crime drama revolves around a group of bank robbers in Boston and the FBI agent determined to bring them down.

3. American Gangster (2007) - Based on a true story, this crime film explores the rise and fall of a Harlem drug lord and the detective who becomes obsessed with bringing him to justice.

4. Heat (1995) - Directed by Michael Mann, this crime thriller features an intense cat-and-mouse game between a skilled detective and a professional thief, both equally dedicated to their respective professions.

5. Training Day (2001) - Denzel Washington won an Academy Award for his portrayal of a corrupt narcotics detective who takes a rookie cop under his wing for a day of questionable police work.

6. The Departed (2006) - Although it's the 

### Try Output Parsing

Let's parse the outputs into a structured Pydantic object.

In [9]:
from typing import List
from pydantic import BaseModel, Field
from llama_index.output_parsers import PydanticOutputParser

class Movie(BaseModel):
    """Object representing a single movie."""
    name: str = Field(..., description="Name of the movie.")
    year: int = Field(..., description="Year of the movie.")

class Movies(BaseModel):
    """Object representing a list of movies."""
    movies: List[Movie] = Field(..., description="List of movies.")


llm = OpenAI(model="gpt-3.5-turbo")
output_parser = PydanticOutputParser(Movies)
prompt_str = """\
Please generate related movies to {movie_name}. Output with the following JSON format: 
"""
prompt_str = prompt_str + output_parser.format_string

In [11]:
# add JSON spec to prompt template
prompt_tmpl = PromptTemplate(prompt_str)

p = QueryPipeline(chain=[prompt_tmpl, llm, output_parser], verbose=True)
output = p.run(movie_name="Toy Story")

> Running module 9e7f0bd6-8d50-4ba5-bbe6-32a33f1a3817 with input: 
movie_name: Toy Story

> Running module b4c80e7c-e145-47e5-86ec-c31d9c517c4e with input: 
messages: Please generate related movies to Toy Story. Output with the following JSON format: 

Here's a JSON schema to follow:
{"title": "Movies", "description": "Object representing a list of movies.", "type"...

> Running module 1756c5a1-405a-417b-80fd-e77021bcb507 with input: 
input: assistant: {
  "movies": [
    {
      "name": "Finding Nemo",
      "year": 2003
    },
    {
      "name": "Monsters, Inc.",
      "year": 2001
    },
    {
      "name": "The Incredibles",
      "y...



In [12]:
output

Movies(movies=[Movie(name='Finding Nemo', year=2003), Movie(name='Monsters, Inc.', year=2001), Movie(name='The Incredibles', year=2004), Movie(name='Cars', year=2006), Movie(name='Ratatouille', year=2007), Movie(name='WALL-E', year=2008), Movie(name='Up', year=2009), Movie(name='Toy Story 2', year=1999), Movie(name='Toy Story 3', year=2010), Movie(name='Toy Story 4', year=2019)])

## Chain Together Query Rewriting Workflow (prompts + LLM) with Retrieval

Here we try a slightly more complex workflow where we send the input through two prompts before initiating retrieval.

1. Generate question about given topic.
2. Hallucinate answer given question, for better retrieval.

Since each prompt only takes in one input, note that the `QueryPipeline` will automatically chain LLM outputs into the prompt and then into the LLM. 

You'll see how to define links more explicitly in the next section.

In [ ]:
from llama_index.postprocessor import CohereRerank

# generate question regarding topic
prompt_str1 = "Please generate a concise question about Paul Graham's life regarding the following topic {topic}"
prompt_tmpl1 = PromptTemplate(prompt_str1)
# use HyDE to hallucinate answer.
prompt_str2 = (
    "Please write a passage to answer the question\n"
    "Try to include as many key details as possible.\n"
    "\n"
    "\n"
    "{query_str}\n"
    "\n"
    "\n"
    'Passage:"""\n'
)
prompt_tmpl2 = PromptTemplate(prompt_str2)

llm = OpenAI(model="gpt-3.5-turbo")
retriever = index.as_retriever(similarity_top_k=5)
p = QueryPipeline(
    chain=[prompt_tmpl1, llm, prompt_tmpl2, llm, retriever], verbose=True
)

In [ ]:
nodes = p.run(topic="college")
len(nodes)

> Running module 815c30fe-b56d-453c-bca9-2e328a55910f with input: 
topic: college

> Running module 4f68b26b-f081-4c30-ac91-1b9d8a60518c with input: 
messages: Please generate a concise question about Paul Graham's life regarding the following topic college

> Running module 8ed6f05b-8170-4403-812f-26e0153601d0 with input: 
query_str: assistant: How did Paul Graham's college experience shape his career and entrepreneurial mindset?

> Running module c4b96e89-9f36-4472-aa95-f65db2560ce6 with input: 
messages: Please write a passage to answer the question
Try to include as many key details as possible.


assistant: How did Paul Graham's college experience shape his career and entrepreneurial mindset?


Pass...

> Running module 1bfa5da7-c112-4f2c-8238-df3df698ce0e with input: 
input: assistant: Paul Graham's college experience played a pivotal role in shaping his career and entrepreneurial mindset. As a student at Cornell University, Graham immersed himself in the world of compute...



5

## Chain Together a Full RAG Pipeline

Here we chain together a full RAG pipeline consisting of query rewriting, retrieval, reranking, and response synthesis.

Here we can't use `chain` syntax because certain modules depend on multiple inputs (for instance, response synthesis expects both the retrieved nodes and the original question). Instead we'll construct a DAG explicitly, through `add_modules` and then `add_link`.

In [ ]:
from llama_index.postprocessor import CohereRerank
from llama_index.response_synthesizers import TreeSummarize
from llama_index import ServiceContext

# define modules
prompt_str = "Please generate a question about Paul Graham's life regarding the following topic {topic}"
prompt_tmpl = PromptTemplate(prompt_str)
llm = OpenAI(model="gpt-3.5-turbo")
retriever = index.as_retriever(similarity_top_k=3)
reranker = CohereRerank()
summarizer = TreeSummarize(
    service_context=ServiceContext.from_defaults(llm=llm)
)

In [ ]:
# define query pipeline
p = QueryPipeline(verbose=True)
p.add_modules(
    {
        "llm": llm,
        "prompt_tmpl": prompt_tmpl,
        "retriever": retriever,
        "summarizer": summarizer,
        "reranker": reranker,
    }
)
p.add_link("prompt_tmpl", "llm")
p.add_link("llm", "retriever")
p.add_link("retriever", "reranker", dest_key="nodes")
p.add_link("llm", "reranker", dest_key="query_str")
p.add_link("reranker", "summarizer", dest_key="nodes")
p.add_link("llm", "summarizer", dest_key="query_str")

In [ ]:
response = p.run(topic="YC")

> Running module prompt_tmpl with input: 
topic: YC

> Running module llm with input: 
messages: Please generate a question about Paul Graham's life regarding the following topic YC

> Running module retriever with input: 
input: assistant: What role did Paul Graham play in the founding and development of Y Combinator (YC)?

> Running module reranker with input: 
query_str: assistant: What role did Paul Graham play in the founding and development of Y Combinator (YC)?
nodes: [NodeWithScore(node=TextNode(id_='543f958b-2c46-4c0f-b046-22e0a60ea950', embedding=None, metadata={'file_path': '../data/paul_graham/paul_graham_essay.txt', 'file_name': 'paul_graham_essay.txt', 'file

> Running module summarizer with input: 
query_str: assistant: What role did Paul Graham play in the founding and development of Y Combinator (YC)?
nodes: [NodeWithScore(node=TextNode(id_='6b43fef2-821a-49a2-a043-9d816b70560f', embedding=None, metadata={'file_path': '../data/paul_graham/paul_graham_essay.txt', 'file_

In [ ]:
print(str(response))

Paul Graham played a significant role in the founding and development of Y Combinator (YC). He was one of the co-founders of YC and was actively involved in its early stages. He helped establish the Summer Founders Program (SFP) and was responsible for selecting and funding the initial batch of startups. Graham also played a key role in shaping the funding model for YC, based on previous deals and agreements. As YC grew, Graham's involvement expanded, and he dedicated a significant amount of his time and attention to the organization. However, in later years, he decided to step back from his role and hand over the leadership to Sam Altman.


## Defining a Custom Component in a Query Pipeline

You can easily define a custom component. Simply subclass a `QueryComponent`, implement validation/run functions + some helpers, and plug it in.

Let's wrap the related movie generation prompt+LLM chain from the first example into a custom component.

In [21]:
from llama_index.query_pipeline import CustomQueryComponent, InputKeys, OutputKeys
from typing import Dict, Any
from llama_index.llms.llm import BaseLLM
from pydantic import Field


class RelatedMovieComponent(CustomQueryComponent):
    """Related movie component."""

    llm: BaseLLM = Field(..., description="OpenAI LLM")

    def _validate_component_inputs(self, input: Dict[str, Any]) -> Dict[str, Any]:
        """Validate component inputs during run_component."""
        # NOTE: this is OPTIONAL but we show you here how to do validation as an example
        return input

    @property
    def _input_keys(self) -> set:
        """Input keys dict."""
        # NOTE: These are required inputs. If you have optional inputs please override 
        # `optional_input_keys_dict`
        return {"movie"}

    @property
    def _output_keys(self) -> set:
        return {"output"}

    def _run_component(self, **kwargs) -> Dict[str, Any]:
        """Run the component."""
        # use QueryPipeline itself here for convenience
        prompt_str = "Please generate related movies to {movie_name}"
        prompt_tmpl = PromptTemplate(prompt_str)
        p = QueryPipeline(chain=[prompt_tmpl, llm])
        return {"output": p.run(movie_name=kwargs["movie"])}

Let's try the custom component out! We'll also add a step to convert the output to Shakespeare.

In [22]:
llm = OpenAI(model="gpt-3.5-turbo")
component = RelatedMovieComponent(llm=llm)

# let's add some subsequent prompts for fun
prompt_str = """\
Here's some text:

{text}

Can you rewrite this in the voice of Shakespeare?
"""
prompt_tmpl = PromptTemplate(prompt_str)

p = QueryPipeline(chain=[component, prompt_tmpl, llm], verbose=True)

In [23]:
output = p.run(movie="Love Actually")

> Running module da1594b5-1b11-4ce7-86e2-e11e2a1988b8 with input: 
movie: Love Actually

> Running module 8c3b70d5-69ff-450c-a8c1-7fb6756b857c with input: 
text: assistant: 1. "Valentine's Day" (2010)
2. "New Year's Eve" (2011)
3. "The Holiday" (2006)
4. "Crazy, Stupid, Love" (2011)
5. "Notting Hill" (1999)
6. "Bridget Jones's Diary" (2001)
7. "Four Weddings a...

> Running module 26d6b4be-488a-47fd-8ecc-449d5af4d4a9 with input: 
messages: Here's some text:

assistant: 1. "Valentine's Day" (2010)
2. "New Year's Eve" (2011)
3. "The Holiday" (2006)
4. "Crazy, Stupid, Love" (2011)
5. "Notting Hill" (1999)
6. "Bridget Jones's Diary" (2001)
...



In [24]:
print(str(output))

assistant: 1. "Valentine's Day" (2010) - "A tale of love, where Cupid's arrows fly,
   And hearts entwined, in passion's sweetest sigh."

2. "New Year's Eve" (2011) - "When the old year fades, and new hope is born,
   A tapestry of lives, on this eve adorned."

3. "The Holiday" (2006) - "Two souls, weary of their own abode,
   Find solace in a holiday, love's ode."

4. "Crazy, Stupid, Love" (2011) - "In love's mad dance, where reason takes no part,
   Hearts entangled, in a web of love's art."

5. "Notting Hill" (1999) - "A humble man, a star's love he did find,
   In Notting Hill, where fate's path intertwined."

6. "Bridget Jones's Diary" (2001) - "A fair maiden's tale, of love's trials and woes,
   In her diary, her heart's secrets she shows."

7. "Four Weddings and a Funeral" (1994) - "Four unions blessed, with joy and mirth,
   Yet death's cold hand, reminds us of life's worth."

8. "About Time" (2013) - "A time-traveling tale, where love's flame does burn,
   And choices made, fo